In [5]:
import tabula
from IPython.display import display
import os
from contextlib import redirect_stderr
import pandas as pd
import numpy as np


# --- Bloco para suprimir o aviso do jpype ---
devnull = open(os.devnull, 'w')
with redirect_stderr(devnull):
    # O tabula.read_pdf retorna uma LISTA de DataFrames.
    # Um para cada tabela encontrada na página.
    lista_de_tabelas = tabula.read_pdf("Planilha _orcamentaria_analitica.pdf", pages="1")
devnull.close()
# --- Fim do bloco de supressão ---

# Verificamos se alguma tabela foi encontrada antes de tentar exibi-la
if lista_de_tabelas:
    print(f"Encontrada(s) {len(lista_de_tabelas)} tabela(s) na página 1.")
    
    # Pega a primeira tabela da lista. Usamos .copy() por segurança.
    df = lista_de_tabelas[0].copy()
    
    print("\n--- 1. DataFrame Original (Extraído pelo Tabula) ---")
    display(df)

    # --- INÍCIO DO TRATAMENTO COM PANDAS ---

    # ETAPA A: Corrigir o cabeçalho do DataFrame
    # A primeira linha de dados (índice 0) contém os nomes corretos das colunas.
    df.columns = df.iloc[0]
    # Agora que a primeira linha virou o cabeçalho, podemos removê-la dos dados.
    df = df.drop(df.index[0])
    # Vamos renomear a primeira coluna que pode ter um nome estranho ('1.1')
    df.rename(columns={df.columns[0]: 'Item'}, inplace=True)
    
    print("\n--- 2. DataFrame Após Correção do Cabeçalho ---")
    display(df)

    # ETAPA B: Juntar as linhas com quebra de texto
    indices_para_remover = []
    for index, row in df.iterrows():
        # A condição para identificar uma linha de continuação é 'Código Banco' ser nulo (NaN)
        if pd.isna(row['Código Banco']):
            # Índice da linha principal (a de cima)
            prev_index = index - 1
            
            # Itera sobre cada célula da linha de continuação
            for col_name, cell_value in row.items():
                if pd.notna(cell_value):
                    valor_anterior = df.at[prev_index, col_name]
                    # Garante que não estamos juntando 'NaN' como texto
                    if pd.isna(valor_anterior):
                        valor_anterior = '' # Trata o NaN anterior como texto vazio
                    
                    # Junta o valor antigo com o novo
                    novo_valor = str(valor_anterior) + ' ' + str(cell_value)
                    df.at[prev_index, col_name] = novo_valor
            
            indices_para_remover.append(index)

    # Remove as linhas de continuação que já foram processadas
    df = df.drop(indices_para_remover)
    # Reseta o índice para ficar sequencial e limpo
    df = df.reset_index(drop=True)

    print("\n--- 3. Resultado Final (DataFrame Limpo e Corrigido) ---")
    display(df)
    print(df)
    # --- FIM DO TRATAMENTO ---

else:
    print("Nenhuma tabela foi encontrada na página especificada.")

Encontrada(s) 4 tabela(s) na página 1.

--- 1. DataFrame Original (Extraído pelo Tabula) ---


,1,Unnamed: 0,BANHEIRO FEMININO,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,"3.980,89"
0,1.1,Código Banco,Descrição,Tipo,Und,Quant.,Valor Unit,Total
1,Composição,85005 SINAPI,"ESPELHO CRISTAL, ESPESSURA 4MM, COM PARAFUSOS ...",ESQV -,m2,"1,0000000","492,58","492,58"
2,NaN,NaN,SEM MOLDURA,ESQUADRIAS/FERRAGENS/VIDR,NaN,NaN,NaN,NaN
3,Composição,88316 SINAPI,SERVENTE COM ENCARGOS COMPLEMENTARES,OSESDI - SERVIÇOS DIVERSOS,H,"0,4000000","22,95","9,18"
4,Auxiliar,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Composição,88325 SINAPI,VIDRACEIRO COM ENCARGOS COMPLEMENTARES,SEDI - SERVIÇOS DIVERSOS,H,"2,0000000","23,11","46,22"
6,Auxiliar,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Insumo,00000442 SINAPI,"PARAFUSO FRANCES M16 EM ACO GALVANIZADO, COMPR...",Material,UN,"4,0000000","7,17","28,68"
8,NaN,NaN,"MM, DIAMETRO = 16 MM, CABECA ABAULADA",NaN,NaN,NaN,NaN,NaN
9,Insumo,00011186 SINAPI,ESPELHO CRISTAL E = 4 MM,Material,M2,"1,0000000","408,50","408,50"



--- 2. DataFrame Após Correção do Cabeçalho ---


,Item,Código Banco,Descrição,Tipo,Und,Quant.,Valor Unit,Total
1,Composição,85005 SINAPI,"ESPELHO CRISTAL, ESPESSURA 4MM, COM PARAFUSOS ...",ESQV -,m2,"1,0000000","492,58","492,58"
2,NaN,NaN,SEM MOLDURA,ESQUADRIAS/FERRAGENS/VIDR,NaN,NaN,NaN,NaN
3,Composição,88316 SINAPI,SERVENTE COM ENCARGOS COMPLEMENTARES,OSESDI - SERVIÇOS DIVERSOS,H,"0,4000000","22,95","9,18"
4,Auxiliar,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Composição,88325 SINAPI,VIDRACEIRO COM ENCARGOS COMPLEMENTARES,SEDI - SERVIÇOS DIVERSOS,H,"2,0000000","23,11","46,22"
6,Auxiliar,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Insumo,00000442 SINAPI,"PARAFUSO FRANCES M16 EM ACO GALVANIZADO, COMPR...",Material,UN,"4,0000000","7,17","28,68"
8,NaN,NaN,"MM, DIAMETRO = 16 MM, CABECA ABAULADA",NaN,NaN,NaN,NaN,NaN
9,Insumo,00011186 SINAPI,ESPELHO CRISTAL E = 4 MM,Material,M2,"1,0000000","408,50","408,50"



--- 3. Resultado Final (DataFrame Limpo e Corrigido) ---


,Item,Código Banco,Descrição,Tipo,Und,Quant.,Valor Unit,Total
0,Composição,85005 SINAPI,"ESPELHO CRISTAL, ESPESSURA 4MM, COM PARAFUSOS ...",ESQV - ESQUADRIAS/FERRAGENS/VIDR,m2,"1,0000000","492,58","492,58"
1,Composição Auxiliar,88316 SINAPI,SERVENTE COM ENCARGOS COMPLEMENTARES,OSESDI - SERVIÇOS DIVERSOS,H,"0,4000000","22,95","9,18"
2,Composição Auxiliar,88325 SINAPI,VIDRACEIRO COM ENCARGOS COMPLEMENTARES,SEDI - SERVIÇOS DIVERSOS,H,"2,0000000","23,11","46,22"
3,Insumo,00000442 SINAPI,"PARAFUSO FRANCES M16 EM ACO GALVANIZADO, COMPR...",Material,UN,"4,0000000","7,17","28,68"
4,Insumo,00011186 SINAPI,ESPELHO CRISTAL E = 4 MM,Material,M2,"1,0000000","408,50","408,50"


0                 Item     Código Banco  \
0           Composição     85005 SINAPI   
1  Composição Auxiliar     88316 SINAPI   
2  Composição Auxiliar     88325 SINAPI   
3               Insumo  00000442 SINAPI   
4               Insumo  00011186 SINAPI   

0                                          Descrição  \
0  ESPELHO CRISTAL, ESPESSURA 4MM, COM PARAFUSOS ...   
1               SERVENTE COM ENCARGOS COMPLEMENTARES   
2             VIDRACEIRO COM ENCARGOS COMPLEMENTARES   
3  PARAFUSO FRANCES M16 EM ACO GALVANIZADO, COMPR...   
4                           ESPELHO CRISTAL E = 4 MM   

0                              Tipo Und     Quant. Valor Unit   Total  
0  ESQV - ESQUADRIAS/FERRAGENS/VIDR  m2  1,0000000     492,58  492,58  
1        OSESDI - SERVIÇOS DIVERSOS   H  0,4000000      22,95    9,18  
2          SEDI - SERVIÇOS DIVERSOS   H  2,0000000      23,11   46,22  
3                          Material  UN  4,0000000       7,17   28,68  
4                          Material  M2  1,0